## 台指日盤當沖策略(Gridsearch)

In [15]:
import polars as pl
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
import os
import glob
import warnings
import gc
from sklearn.model_selection import ParameterGrid
warnings.filterwarnings("ignore")

# =============================================================================
# 設定區
# =============================================================================
START_DATE = "2024-01-01"
END_DATE   = "2024-12-31"
SAVE_DIR   = r"C:\Users\USER\Desktop\雲端同步\Shioaji\Backtesting"
PATTERN    = os.path.join(SAVE_DIR, "stock_kbars_*.parquet")

stock_number = 5

STOCKS_TO_FETCH = ["2330", "2454", "2317", "2881", "2412", "2382", "2308", "2882", "2891", "3711"]

bsr_long_grid   = np.round(np.arange(1.1, 1.21, 0.1), 2)
bsr_short_grid  = np.round(np.arange(0.8, 0.91, 0.1), 2)
vr_high_grid    = np.round(np.arange(1.5, 1.61, 0.1), 2)
vr_midh_grid    = np.round(np.arange(1.2, 1.31, 0.1), 2)
vr_mid_grid     = np.round(np.arange(0.7, 0.81, 0.1), 2)
vr_midlow_grid  = np.round(np.arange(0.5, 0.61, 0.1), 2)
tp_k_grid       = np.arange(1.5, 2.1, 0.5)
sl_k_grid       = np.arange(3.5, 4.1, 0.5)

param_grid = {
    'BSR_THRESHOLD_LONG': bsr_long_grid,
    'BSR_THRESHOLD_SHORT': bsr_short_grid,
    'vr_high': vr_high_grid,
    'vr_mid_high': vr_midh_grid,
    'vr_mid': vr_mid_grid,
    'vr_mid_low': vr_midlow_grid,
    'tp_k': tp_k_grid,
    'sl_k': sl_k_grid
}

start_dt = datetime.strptime(START_DATE, "%Y-%m-%d").date()
end_dt   = datetime.strptime(END_DATE, "%Y-%m-%d").date()

# =============================================================================
# 1. 取最近 21 個交易日（含當日）→ 計算「前 20 日不含當日」的振幅統計
# =============================================================================
dates_df_full = (
    pl.scan_parquet(os.path.join(SAVE_DIR, "txf_kbars.parquet"))
    .with_columns(pl.col("ts").dt.date().alias("date"))
    .select("date")
    .filter(pl.col("date") <= end_dt)
    .unique().sort("date")
    .collect()
)

analysis_dates_df = dates_df_full.filter(
    (pl.col("date") >= start_dt) & (pl.col("date") <= end_dt)
)

pre_window_dates = (
    dates_df_full
      .filter(pl.col("date") < start_dt)
      .tail(40)
)

dates_df = pl.concat([pre_window_dates, analysis_dates_df])

start_trading   = dates_df["date"][0]
analysis_dates  = analysis_dates_df["date"]

kbars_lazy = (
    pl.scan_parquet(os.path.join(SAVE_DIR, "txf_kbars.parquet"))
    .rename({"Open":"open","High":"high","Low":"low","Close":"close","Volume":"volume"})
    .with_columns([
        pl.col("ts").cast(pl.Datetime("us")), 
        pl.col("ts").dt.date().alias("date"),
        pl.col("ts").dt.hour().alias("hour"),
        pl.col("ts").dt.minute().alias("minute"),
    ])
    .filter(
        (pl.col("date") >= start_trading) &
        (pl.col("date") <= end_dt) &
        (
            ((pl.col("hour") == 8)  & (pl.col("minute") >= 45)) |
            ((pl.col("hour") > 8)   & (pl.col("hour") < 13)) |
            ((pl.col("hour") == 13) & (pl.col("minute") <= 45))
        )
    )
    .drop(["hour", "minute"])
    .sort(["date","ts"])
)

minute_close_lazy = (
    kbars_lazy
    .with_columns(pl.col("ts").dt.truncate("1m").alias("minute_ts"))
    .group_by(["date", "minute_ts"])
    .agg(pl.col("close").last().alias("close"))
    .rename({"minute_ts": "ts"})
    .sort(["date","ts"])
    .with_columns([
        pl.col("close").forward_fill().over("date")
    ])
    .lazy()
)

daily_range_lazy = (
    kbars_lazy
    .group_by("date")
    .agg([
        pl.col("high").max().alias("high_max"),
        pl.col("low").min().alias("low_min"),
    ])
    .with_columns([
        (pl.col("high_max") - pl.col("low_min")).alias("amplitude")
    ])
    .sort("date")
    .with_columns([
        pl.col("amplitude").shift(1).alias("amp_prev"),
        pl.col("amplitude").shift(1).rolling_max(window_size=20).alias("max"),
        pl.col("amplitude").shift(1).rolling_mean(window_size=20).alias("avg"),
        pl.col("amplitude").shift(1).rolling_min(window_size=20).alias("min"),
    ])
    .with_columns([
        pl.col("amp_prev").rank(method="average").over(pl.lit(1)).alias("rank")
    ])
    .with_columns([
        pl.when(pl.col("rank") > 10)
          .then(None)
          .otherwise(pl.col("amp_prev"))
          .rolling_mean(window_size=20)
          .alias("top"),
        pl.when(pl.col("rank") <= (pl.len().over(pl.lit(1)) - 10))
          .then(pl.col("amp_prev"))
          .otherwise(None)
          .rolling_mean(window_size=20)
          .alias("bot")
    ])
    .select(["date", "amp_prev", "max", "avg", "min", "top", "bot", "amplitude"])
)

daily_range_df = daily_range_lazy.collect().to_pandas()
daily_range_df["top"] = daily_range_df["amp_prev"].rolling(20, min_periods=1) \
    .apply(lambda x: pd.Series(x).nlargest(10).mean(), raw=False)
daily_range_df["bot"] = daily_range_df["amp_prev"].rolling(20, min_periods=1) \
    .apply(lambda x: pd.Series(x).nsmallest(10).mean(), raw=False)
daily_range_df = daily_range_df.drop(columns=["rank"], errors="ignore")

daily_range_lazy = (
    pl.from_pandas(daily_range_df)
      .with_columns([
          pl.col("date").cast(pl.Date),
          pl.col("max"),
          pl.col("avg"),
          pl.col("min"),
          pl.col("top"),
          pl.col("bot"),
      ])
      .lazy()
)

del daily_range_df
gc.collect()

# =============================================================================
# 2. 取最近 21 個交易日的 ticks → 每分鐘累積 vs 過去 20 日 KBARS 平均累積 → volume_ratio
# =============================================================================
kbars_min_cum = (
    kbars_lazy
    .with_columns(pl.col("ts").dt.truncate("1m").alias("minute_ts"))
    .group_by(["date", "minute_ts"])
    .agg(pl.col("volume").sum().alias("min_vol_k"))
    .sort(["date", "minute_ts"])
    .with_columns([
        pl.col("min_vol_k").cum_sum().over("date").alias("cum_vol_k"),
        pl.col("minute_ts").dt.strftime("%H:%M").alias("minute_str")
    ])
)

kbars_avg_cum = (
    kbars_min_cum
    .with_columns(
        pl.col("cum_vol_k")
          .shift(1)
          .rolling_mean(20)
          .over("minute_str")
          .alias("avg_cum_vol")
    )
    .group_by("minute_str")
    .agg(pl.col("avg_cum_vol").last())
)

volume_ratio_lazy = (
    kbars_min_cum
    .join(kbars_avg_cum, on="minute_str", how="left")
    .with_columns([
        (pl.col("cum_vol_k") / pl.col("avg_cum_vol")).alias("volume_ratio")
    ])
    .with_columns(
        pl.col("volume_ratio").fill_null(0).forward_fill().over("date")
    )
    .select([
        pl.col("date"),
        pl.col("minute_ts"),
        pl.col("cum_vol_k"),
        pl.col("avg_cum_vol"),
        pl.col("volume_ratio")
    ])
    .rename({
        "minute_ts": "ts",
        "cum_vol_k": "cum_vol"
    })
    .lazy()
)

del kbars_min_cum, kbars_avg_cum
gc.collect()

# =============================================================================
# 3. 每分鐘內外盤成交比 bsr
# =============================================================================
ticks_base_lazy = (
    pl.scan_parquet(os.path.join(SAVE_DIR, "txf_ticks.parquet"))
    .with_columns([
        pl.col("ts").cast(pl.Datetime("us")).alias("ts"),
        pl.col("ts").dt.date().alias("date"),
        pl.col("volume"),
        pl.col("tick_type")
    ])
    .filter(
        pl.col("ts").dt.time().is_between(time(8,45), time(13,45)) &
        pl.col("tick_type").is_in([1,2])
    )
    .sort(["date","ts"])
    .with_columns([
        pl.arange(0, pl.len()).over("date").alias("row_idx")
    ])
    .filter(pl.col("row_idx") > 0)
    .drop("row_idx")
    .lazy()
)

daily_bsr_lazy = (
    ticks_base_lazy
    .with_columns([
        pl.when(pl.col("tick_type")==1)
          .then(pl.col("volume")).otherwise(0)
          .cum_sum().over("date").alias("outer_cum"),
        pl.when(pl.col("tick_type")==2)
          .then(pl.col("volume")).otherwise(0)
          .cum_sum().over("date").alias("inner_cum"),
    ])
    .with_columns([
        pl.when(pl.col("inner_cum")==0)
          .then(None)
          .otherwise(pl.col("outer_cum")/pl.col("inner_cum"))
          .alias("bsr")
    ])
    .with_columns(pl.col("ts").dt.truncate("1m").alias("minute_ts"))
    .group_by(["date","minute_ts"])
    .agg(pl.col("bsr").last())
    .sort(["date","minute_ts"])
    .rename({"minute_ts":"ts"})
    .with_columns(pl.col("bsr").forward_fill().over("date"))
)

del ticks_base_lazy
gc.collect()

# =============================================================================
# 4. 計算權值股紅K比例 + TSMC 判斷
# =============================================================================
parquet_paths = glob.glob(PATTERN)
stock_lfs = []
for path in parquet_paths:
    sid = os.path.basename(path).split("_")[-1].replace(".parquet", "")
    lf = (
        pl.read_parquet(path).lazy()
        .rename({"Open": "open", "Close": "close"})
        .with_columns([
            pl.col("ts").cast(pl.Datetime),
            pl.col("ts").dt.date().alias("date"),
            pl.lit(sid).alias("sid")
        ])
    )
    stock_lfs.append(lf)

stock_lazy = (
    pl.concat(stock_lfs)
    .filter(pl.col("sid").is_in(STOCKS_TO_FETCH))
)

opening_price = (
    stock_lazy
    .filter(pl.col("ts").dt.time() >= datetime.strptime("09:00", "%H:%M").time())
    .sort(["sid", "date", "ts"])
    .group_by(["sid", "date"])
    .agg(pl.col("open").first().alias("day_open"))
)

minute_close = (
    stock_lazy
    .with_columns(pl.col("ts").dt.truncate("1m").alias("minute_ts"))
    .group_by(["sid","date","minute_ts"])
    .agg(pl.col("close").last().alias("minute_close"))
)

minute_status_lazy = (
    minute_close
    .join(opening_price, on=["sid","date"])
    .with_columns([
        pl.col("minute_ts").alias("ts"),
        (pl.col("minute_close") > pl.col("day_open")).alias("is_red")
    ])
)

tsmc_status_lazy = (
    minute_status_lazy
    .filter(pl.col("sid") == "2330")
    .select(["date","ts","is_red"])
    .rename({"is_red":"tsmc_red"})
)

summary_lazy = (
    minute_status_lazy
    .group_by(["date","ts"])
    .agg(pl.col("is_red").sum().alias("is_red"))
    .join(tsmc_status_lazy, on=["date","ts"], how="left")
    .with_columns([
        pl.col("is_red").forward_fill().over("date"),
        pl.col("tsmc_red").forward_fill().over("date")
    ])
)

del stock_lfs, stock_lazy, opening_price, minute_close, minute_status_lazy, tsmc_status_lazy
gc.collect()

# =============================================================================
# 5. 合併所有指標
# =============================================================================
joined_all = (
    volume_ratio_lazy
    .join(daily_range_lazy, on="date", how="left")
    .join(daily_bsr_lazy, on=["date","ts"], how="left")
    .join(minute_close_lazy, on=["date","ts"], how="left")
    .join(summary_lazy, on=["date","ts"], how="left")
    .with_columns(pl.col("ts").cast(pl.Datetime("us")))
    .filter(pl.col("ts").dt.time().is_between(time(8,45), time(13,45)))
)

float_cols = [
    "cum_vol","avg_cum_vol","volume_ratio",
    "amp_prev","max","avg","min","top","bot","amplitude",
    "bsr","close"
]
int_cols = ["is_red"]
bool_cols = ["tsmc_red"]

joined_filled = (
    joined_all
    .with_columns([
        pl.col(c)
          .forward_fill().over("date")
          .fill_null(0.0)
          .alias(c)
        for c in float_cols
    ] + [
        pl.col(c)
          .forward_fill().over("date")
          .fill_null(0)
          .alias(c)
        for c in int_cols
    ] + [
        pl.col(c)
          .forward_fill().over("date")
          .fill_null(False)
          .alias(c)
        for c in bool_cols
    ])
)

joined_lazy = joined_filled.filter(
    pl.col("date").is_between(
        datetime.strptime(START_DATE, "%Y-%m-%d").date(),
        datetime.strptime(END_DATE, "%Y-%m-%d").date()
    )
)

del joined_all, joined_filled
gc.collect()

# =============================================================================
# 6. 準備 future ticks 資料（用於動態 TP/SL 出場檢查）
# =============================================================================
future_base = (
    pl.scan_parquet(f"{SAVE_DIR}/txf_ticks.parquet")
    .with_columns([
        pl.col("ts").cast(pl.Datetime("us")).alias("future_time"),
        pl.col("ts").dt.date().alias("date"),
    ])
    .filter(
        pl.col("date").is_between(start_trading, end_dt) &
        (pl.col("future_time").dt.time() >= time(8, 45)) &
        (pl.col("future_time").dt.time() <= time(13, 45))
    )
    .select(["future_time", "close", "date"])
    .rename({"close": "future_close"})
)

future_lazy = (
    future_base
    .join_asof(
        joined_lazy
          .with_columns(pl.col("ts").alias("minute_ts"))
          .select([
              pl.col("date"),
              pl.col("minute_ts"),
              pl.col("bsr"), pl.col("volume_ratio"),
              pl.col("max"), pl.col("top"), pl.col("avg"),
              pl.col("bot"), pl.col("min")
          ])
          .sort(["date", "minute_ts"]),
        left_on="future_time",
        right_on="minute_ts",
        by="date",
        strategy="backward",
        tolerance=timedelta(minutes=2)
    )
    .sort(["date", "future_time"])
    .with_columns(
        pl.col("volume_ratio").forward_fill().over("date")
    )
    .with_columns(pl.col("future_close").round(0).cast(pl.Int32))
)

future_df = future_lazy.collect().to_pandas()
future_df = future_df[future_df["future_close"].notna()]

del future_base, future_lazy
gc.collect()

# =============================================================================
# 7. GridSearch 主迴圈 - 分塊執行
# =============================================================================
CHUNK_SIZE = 10
output_path = os.path.join(SAVE_DIR, "grid_search_results.csv")

header_written = os.path.exists(output_path)

param_list = list(ParameterGrid(param_grid))
total_params = len(param_list)
print(f"總參數組合數：{total_params}")

processed_params = set()
if header_written:
    try:
        existing_results = pd.read_csv(output_path)
        param_cols = list(param_grid.keys())
        missing_cols = [col for col in param_cols if col not in existing_results.columns]
        if missing_cols:
            print(f"CSV 檔案缺少欄位：{missing_cols}，將從頭開始")
            processed_params = set()
        else:
            for _, row in existing_results[param_cols].iterrows():
                try:
                    param_tuple = tuple(
                        round(float(val), 2) if isinstance(val, (float, np.floating, str)) and pd.notna(val) else val
                        for val in row
                    )
                    processed_params.add(param_tuple)
                except (ValueError, TypeError) as e:
                    print(f"無法處理 CSV 行：{row}，錯誤：{e}")
                    continue
            print(f"已從 output_path 載入 {len(processed_params)} 個已處理的參數組合")
            print(f"前 5 個已處理的參數組合：{list(processed_params)[:5]}")
    except Exception as e:
        print(f"無法讀取現有結果檔案：{e}，將從頭開始")

unprocessed_params = []
for params in param_list:
    param_tuple = tuple(
        round(float(params[key]), 2) if isinstance(params[key], (float, np.floating)) else params[key]
        for key in param_cols
    )
    if param_tuple not in processed_params:
        unprocessed_params.append(params)

total_unprocessed = len(unprocessed_params)
print(f"剩餘未處理的參數組合數：{total_unprocessed}")

if total_unprocessed == 0:
    print("所有參數組合均已處理，無需繼續執行。")
else:
    for chunk_start in range(0, total_unprocessed, CHUNK_SIZE):
        chunk_end = min(chunk_start + CHUNK_SIZE, total_unprocessed)
        chunk_params = unprocessed_params[chunk_start:chunk_end]
        chunk_results = []
    
        print(f"正在處理參數塊 {chunk_start // CHUNK_SIZE + 1} ({chunk_start} 到 {chunk_end})...")
    
        for params in chunk_params:
            BSR_THRESHOLD_LONG = params['BSR_THRESHOLD_LONG']
            BSR_THRESHOLD_SHORT = params['BSR_THRESHOLD_SHORT']
            VOLUME_RATIO_LEVELS = {
                "high": params['vr_high'],
                "mid_high": params['vr_mid_high'],
                "mid": params['vr_mid'],
                "mid_low": params['vr_mid_low'],
            }
            TP_K = params['tp_k']
            SL_K = params['sl_k']
    
            kbars_seg = (
                kbars_lazy
                .filter(pl.col("ts").dt.time() >= time(8, 45))
                .with_columns([
                    pl.col("high").cum_max().over("date").alias("seg_high"),
                    pl.col("low").cum_min().over("date").alias("seg_low"),
                ])
                .select(["date", "ts", "seg_high", "seg_low"])
            )
    
            entry_candidates = (
                joined_lazy
                .filter(~pl.col("ts").dt.time().is_between(time(8, 45), time(9, 0)))
                .filter(pl.col("bsr").is_not_null())
                .filter(
                    (
                        (pl.col("tsmc_red") == True) &
                        (pl.col("is_red") > stock_number) &
                        (pl.col("bsr") > BSR_THRESHOLD_LONG)
                    ) |
                    (
                        (pl.col("tsmc_red") == False) &
                        (pl.col("is_red") < stock_number) &
                        (pl.col("bsr") < BSR_THRESHOLD_SHORT)
                    )
                )
                .with_columns([
                    pl.col("ts").alias("entry_time"),
                    pl.col("close").alias("entry"),
                    pl.when(pl.col("tsmc_red") & (pl.col("is_red") > stock_number))
                    .then(pl.lit("long"))
                    .otherwise(pl.lit("short"))
                    .alias("side"),
                    pl.col("volume_ratio"),
                    pl.col("bsr"),
                ])
                .sort("entry_time")
            )
    
            entry_lazy = (
                entry_candidates
                .with_columns([
                    pl.when(pl.col("side") == "long")
                    .then(pl.col("bsr"))
                    .otherwise(None)
                    .cum_max()
                    .shift(1)
                    .alias("prev_max_long"),
                    pl.when(pl.col("side") == "short")
                    .then(pl.col("bsr"))
                    .otherwise(None)
                    .cum_min()
                    .shift(1)
                    .alias("prev_min_short"),
                ])
                .filter(
                    (
                        (pl.col("side") == "long") &
                        (
                            pl.col("prev_max_long").is_null() |
                            (pl.col("bsr") > pl.col("prev_max_long"))
                        )
                    ) |
                    (
                        (pl.col("side") == "short") &
                        (
                            pl.col("prev_min_short").is_null() |
                            (pl.col("bsr") < pl.col("prev_min_short"))
                        )
                    )
                )
                .drop(["prev_max_long", "prev_min_short"])
                .join_asof(
                    kbars_seg.sort(["date", "ts"]),
                    left_on="entry_time",
                    right_on="ts",
                    by="date",
                    strategy="backward"
                )
                .with_columns(
                    (pl.col("seg_high") - pl.col("seg_low")).alias("seg_amp")
                )
                .filter(pl.col("seg_amp") <= pl.col("avg"))
                .with_columns(
                    pl.arange(0, pl.len()).over("date").alias("seq")
                )
                .filter(pl.col("seq") < 3)
                .drop("seq")
            )
    
            entry_df = entry_lazy.collect().to_pandas()
    
            del kbars_seg, entry_candidates, entry_lazy
            gc.collect()
    
            exit_records = []
    
            def compute_tp_sl_offset(vol, max_, top, avg, bot, min_):
                if vol > VOLUME_RATIO_LEVELS["high"]:
                    return (max_ + top) / TP_K, (max_ + top) / SL_K
                elif vol > VOLUME_RATIO_LEVELS["mid_high"]:
                    return (top + avg) / TP_K, (top + avg) / SL_K
                elif vol > VOLUME_RATIO_LEVELS["mid"]:
                    return avg , avg / 2
                elif vol > VOLUME_RATIO_LEVELS["mid_low"]:
                    return (avg + bot) / TP_K, (avg + bot) / SL_K
                else:
                    return (bot + min_) / TP_K, (bot + min_) / SL_K
    
            for _, row in entry_df.iterrows():
                last_vol = None
                date = row["date"]
                entry_time = row["entry_time"]
                entry_price = row["entry"]
                side = row["side"]
                cutoff_time = pd.Timestamp(f"{date} 13:30:00")
                top_amp = row["top"]
                seg_high = row["seg_high"]
                seg_low = row["seg_low"]
                seg_amp = row["seg_amp"]
    
                fut_ticks = future_df[
                    (future_df["date"] == date) &
                    (future_df["future_time"] > entry_time) &
                    (future_df["future_time"] <= cutoff_time)
                ]
    
                exit_time, exit_price, exit_type = None, None, "Close"
                tp_pts, sl_pts = None, None
                long_tp, long_sl, short_tp, short_sl = None, None, None, None
    
                if fut_ticks.empty:
                    continue
                else:
                    for _, tick in fut_ticks.iterrows():
                        vol = tick["volume_ratio"]
                        if pd.isna(vol):
                            vol = last_vol
                        else:
                            last_vol = vol
    
                        if vol is None:
                            continue
    
                        tp_offset, sl_offset = compute_tp_sl_offset(
                            vol, tick["max"], tick["top"],
                            tick["avg"], tick["bot"], tick["min"]
                        )
                        if pd.isna(tp_offset) or pd.isna(sl_offset):
                            continue
                        if seg_amp >= (top_amp / 2):
                            tp_offset /= 2
                            sl_offset /= 2
    
                        tp_pts, sl_pts = int(round(tp_offset)), int(round(sl_offset))
    
                        if side == "long":
                            tp_price = entry_price + tp_offset
                            sl_price = entry_price - sl_offset
                            long_tp, long_sl = int(round(tp_price)), int(round(sl_price))
                            if tick["future_close"] >= tp_price:
                                exit_time, exit_price, exit_type = tick["future_time"], tp_price, "TP"
                                break
                            elif tick["future_close"] <= sl_price:
                                exit_time, exit_price, exit_type = tick["future_time"], sl_price, "SL"
                                break
                        else:
                            tp_price = entry_price - tp_offset
                            sl_price = entry_price + sl_offset
                            short_tp, short_sl = int(round(tp_price)), int(round(sl_price))
                            if tick["future_close"] <= tp_price:
                                exit_time, exit_price, exit_type = tick["future_time"], tp_price, "TP"
                                break
                            elif tick["future_close"] >= sl_price:
                                exit_time, exit_price, exit_type = tick["future_time"], sl_price, "SL"
                                break
    
                    if exit_time is None:
                        last_tick = fut_ticks.iloc[-1]
                        tp_offset, sl_offset = compute_tp_sl_offset(
                            last_tick["volume_ratio"], last_tick["max"], last_tick["top"],
                            last_tick["avg"], last_tick["bot"], last_tick["min"]
                        )
                        if pd.isna(tp_offset) or pd.isna(sl_offset):
                            continue
                        tp_pts, sl_pts = int(round(tp_offset)), int(round(sl_offset))
                        exit_time = last_tick["future_time"]
                        exit_price = last_tick["future_close"]
                        if side == "long":
                            long_tp = int(round(entry_price + tp_offset))
                            long_sl = int(round(entry_price - sl_offset))
                        else:
                            short_tp = int(round(entry_price - tp_offset))
                            short_sl = int(round(entry_price + sl_offset))
    
                pnl = exit_price - entry_price if side == "long" else entry_price - exit_price
    
                exit_records.append({
                    "entry_time": entry_time,
                    "exit_time": exit_time,
                    "type": exit_type,
                    "date": date,
                    "side": side,
                    "hold": entry_price,
                    "long_tp": long_tp if side == "long" else None,
                    "long_sl": long_sl if side == "long" else None,
                    "short_tp": short_tp if side == "short" else None,
                    "short_sl": short_sl if side == "short" else None,
                    "pnl": int(round(pnl)),
                    "tp_pts": tp_pts,
                    "sl_pts": sl_pts,
                    "vol%": round(row["volume_ratio"], 2),
                    "bsr": round(row["bsr"], 2),
                })
    
            del entry_df, fut_ticks
            gc.collect()
    
            if not exit_records:
                chunk_results.append({
                    **params,
                    'trade_count': 0,
                    'final_cumpnl': 0
                })
                continue
    
            exit_df = pd.DataFrame(exit_records)
            exit_df["cumpnl"] = exit_df["pnl"].cumsum()
    
            range_df = daily_range_lazy.collect().to_pandas()
            range_df = range_df[["date", "max", "top", "avg", "bot", "min", "amplitude"]]
            exit_df = exit_df.merge(range_df, on="date", how="left")
    
            red_df = (
                summary_lazy
                .select(["date", "ts", "is_red"])
                .collect()
                .rename({"ts": "entry_time"})
                .to_pandas()
            )
    
            exit_df["entry_time_str"] = pd.to_datetime(exit_df["entry_time"]).dt.strftime("%H:%M")
            red_df["entry_time_str"] = pd.to_datetime(red_df["entry_time"]).dt.strftime("%H:%M")
            exit_df = exit_df.merge(red_df[["date", "entry_time_str", "is_red"]], on=["date", "entry_time_str"], how="left")
    
            if "is_red" in exit_df.columns:
                cols = exit_df.columns.tolist()
                cols.insert(cols.index("bsr") + 1, cols.pop(cols.index("is_red")))
                exit_df = exit_df[cols]
    
            exit_df["entry"] = pd.to_datetime(exit_df["entry_time"]).dt.strftime("%H:%M")
            exit_df["exit"] = pd.to_datetime(exit_df["exit_time"]).dt.strftime("%H:%M")
    
            final_cols = [
                "date", "entry", "exit", "type", "side", "hold",
                "long_tp", "long_sl", "short_tp", "short_sl",
                "bsr", "is_red", "vol%", "max", "top", "avg", "bot", "min", "amplitude",
                "tp_pts", "sl_pts", "pnl", "cumpnl"
            ]
            exit_df = exit_df[final_cols]
    
            for col in ["max", "top", "avg", "bot", "min", "amplitude"]:
                exit_df[col] = exit_df[col].round(0).astype("Int32")
            for col in ["long_tp", "long_sl", "short_tp", "short_sl"]:
                exit_df[col] = exit_df[col].dropna().round(0).astype("Int32")
    
            final_cumpnl = exit_df["cumpnl"].iloc[-1] if not exit_df.empty else 0
            chunk_results.append({
                **params,
                'trade_count': len(exit_df),
                'final_cumpnl': final_cumpnl
            })
    
            del exit_df, range_df, red_df, exit_records
            gc.collect()
    
        if chunk_results:
            chunk_results_df = pd.DataFrame(chunk_results)
            chunk_results_df.to_csv(
                output_path,
                mode="a",
                header=not header_written,
                index=False
            )
            header_written = True
            print(f"塊 {chunk_start // CHUNK_SIZE + 1} 的結果已儲存至 {output_path}")
    
        del chunk_results, chunk_results_df
        gc.collect()

# =============================================================================
# 8. 輸出最終 GridSearch 結果
# =============================================================================
if os.path.exists(output_path):
    grid_results_df = pd.read_csv(output_path)
    if not grid_results_df.empty:
        best_params = grid_results_df.loc[grid_results_df['final_cumpnl'].idxmax()]
        print("最佳參數組合：")
        print(best_params)
        print("\n所有參數組合結果：")
        print(grid_results_df.sort_values(by='final_cumpnl', ascending=False))
        print(f"\nGridSearch 結果已儲存至 output_path")
    else:
        print("無任何交易訊號生成。")
    del grid_results_df
else:
    print("無任何交易訊號生成，輸出檔案未生成。")

gc.collect()

總參數組合數：256
已從 output_path 載入 256 個已處理的參數組合
前 5 個已處理的參數組合：[(1.1, 0.9, 1.5, 1.3, 0.7, 0.5, 1.5, 4.0), (1.1, 0.8, 1.5, 1.3, 0.7, 0.6, 1.5, 3.5), (1.2, 0.9, 1.5, 1.3, 0.8, 0.5, 1.5, 3.5), (1.2, 0.8, 1.5, 1.3, 0.7, 0.5, 1.5, 3.5), (1.2, 0.9, 1.5, 1.3, 0.7, 0.6, 2.0, 3.5)]
剩餘未處理的參數組合數：0
所有參數組合均已處理，無需繼續執行。
最佳參數組合：
BSR_THRESHOLD_LONG        1.1
BSR_THRESHOLD_SHORT       0.9
sl_k                      4.0
tp_k                      1.5
vr_high                   1.6
vr_mid                    0.8
vr_mid_high               1.2
vr_mid_low                0.5
trade_count             115.0
final_cumpnl           1733.0
Name: 108, dtype: float64

所有參數組合結果：
     BSR_THRESHOLD_LONG  BSR_THRESHOLD_SHORT  sl_k  tp_k  vr_high  vr_mid  \
108                 1.1                  0.9   4.0   1.5      1.6     0.8   
109                 1.1                  0.9   4.0   1.5      1.6     0.8   
101                 1.1                  0.9   4.0   1.5      1.5     0.8   
100                 1.1                  0.9  

0